## Embeddings con OpenAI
 Embedding en procesamiento de lenguaje natural es una técnica que se usa para representar palabras como vectores numéricos en un espacio n-dimensional, donde n es el número de características que se quieran representar. Se busca que estas representaciones capturen el significado semántico y sintáctico de las palabras y, por lo tanto, que palabras similares tengan vectores similares. 
 
 Estos embeddings se utilizan después como entrada en modelos de aprendizaje automático para realizar tareas de procesamiento de lenguaje natural, como clasificación de texto o traducción automática.

In [ ]:
# Antes de comenzar importamos las dependencias necesarias

import openai # Librería de OpenAI para acceso a su API
import gradio as gr # Librería para crear la interfaz gráfica
import pandas as pd # Librería para trabajar con DataFrames en Python

from openai.embeddings_utils import get_embedding # Función para obtener la representación vectorial de un texto
from openai.embeddings_utils import cosine_similarity # Función para calcular la similitud coseno entre dos embeddings


In [ ]:
# Definimos la API Key para vincular el cuaderno con nuestra cuenta de OpenAI
openai.api_key = ""

## Cómo usar embeddings
Al hacer embedding de un dato, lo estamos convirtiendo a un vector numérico, datos similares estarán más cercanos entre si cuando semanticamente son similares.

Como ejemplo, en una lista de palabras

In [ ]:
# Definimos una lista
palabras = ["casa", "perro", "gato", "lobo", "leon", "zebra", "tigre"]

In [ ]:
# Creamos un diccionario vacío
diccionario = {}
# Iteramos a través de una lista de palabras
for i in palabras:
    diccionario[i] = get_embedding(i, engine="text-embedding-ada-002")

In [ ]:
# Se llama a la función keys() del diccionario para obtener una vista iterable de sus claves.
diccionario.keys()

Ahora lo vemos con una palabra en específico

In [ ]:
# Define la palabra 'gato'
palabra = "perro"

# Imprime los primeros 10 valores del diccionario asociado a la palabra 'gato'
print("Primeros 10 valores de {}:\n".format(palabra), diccionario[palabra][:10])

# Imprime un salto de línea
print("\n")

# Imprime el número de dimensiones del dato embebido
print("Número de dimensiones del dato embebido", len(diccionario[palabra]))


## Comparar dos embeddings
Debido a que los embeddings son una representacion vectorial de los datos en un espacio latente, podemos medir la distancia entre dos vectores y asi obtener que tan similares son. Podemos comparar una palabra nueva o alguna de las que ya fueron embebidas.

In [ ]:
# Definimos la palabra nueva a comparar
n_palabra = "agujero negro"

# Definimos la palabra en el diccionario con la que compararemos la nueva palabra
palabra_comparar = "perro"

# Obtenemos un vector de embedding de la nueva palabra, utilizando el motor de "text-embedding-ada-002"
n_palabra_embed = get_embedding(n_palabra, engine="text-embedding-ada-002")

# Obtenemos la similitud coseno entre el vector de embedding de la palabra a comparar en el diccionario y el vector de embedding de la nueva palabra
similitud = cosine_similarity(diccionario[palabra_comparar], n_palabra_embed)

# Imprimimos la similitud coseno para mostrar el resultado
print(similitud)


## Sumar embeddings
Como los vectores contienen valores numericos, podemos sumarlos y el resultado será un nuevo vector de un concepto que una los elementos sumados.

In [ ]:
# Sumar dos listas usando pandas
sumados = (pd.DataFrame(diccionario["leon"])) + (pd.DataFrame(diccionario["zebra"]))

# Calcular la longitud de la lista sumada
len(sumados)

# Calcular la similitud coseno entre cada lista del diccionario y la lista sumada
for key, value in diccionario.items():
    print(key, ":", cosine_similarity(diccionario[key], sumados))


## Aplicado en un Chatbot

Usaremos Gradio para hacer una interfaz básica donde podremos hacer preguntas y obtendremos una respuesta.

In [ ]:
# Crea embeddings para cada texto en el archivo CSV

# Se define la función `embed_text` con un argumento opcional path igual a "texto.csv"
def embed_text(path="texto.csv"):
    # Se lee el archivo csv ubicado en `path` y se guarda en un DataFrame llamado `conocimiento_df`
    conocimiento_df = pd.read_csv(path)
    
    # Se agrega una columna `Embedding` al DataFrame, que se rellena utilizando el método `apply` y
    # una función lambda que llama a otra función llamada `get_embedding` con el motor de embedding 'text-embedding-ada-002'
    conocimiento_df['Embedding'] = conocimiento_df['texto'].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))
    
    # Se guarda el DataFrame con la nueva columna en un nuevo archivo csv 'embeddings.csv'
    conocimiento_df.to_csv('embeddings.csv')
    
    # Se devuelve el DataFrame `conocimiento_df`
    return conocimiento_df


# Busca los textos más similares a una búsqueda dada y devuelve los resultados junto con los embeddings correspondientes.

# Definición de la función 'buscar'
def buscar(busqueda, datos, n_resultados=5):
    # Obtención del vector de incrustación (embedding) de la búsqueda usando el motor 'text-embedding-ada-002'
    busqueda_embed = get_embedding(busqueda, engine="text-embedding-ada-002")
    
    # Cálculo de la similitud coseno entre el embedding de la búsqueda y cada uno de los embeddings de los datos.
    datos["Similitud"] = datos['Embedding'].apply(lambda x: cosine_similarity(x, busqueda_embed))
    
    # Ordenar los datos según la similitud, de mayor a menor.
    datos = datos.sort_values("Similitud", ascending=False)
    
    # Selección de los 'n_resultados' datos más similares, y regresados los campos: 'texto', 'Similitud', 'Embedding'.
    return datos.iloc[:n_resultados][["texto", "Similitud", "Embedding"]]

# Llamamos la funcion con un archivo csv como argumento y almacenamos los resultados en "texto_emb"
texto_emb = embed_text("./chatbot_qa.csv")

### Crea una interfaz de usuario para buscar en el archivo CSV

# Define los elementos de la interfaz (cuadro de búsqueda, botón y resultados)
with gr.Blocks() as demo:
    busqueda = gr.Textbox(label="Buscar")
    output = gr.DataFrame(headers=['texto'])
    greet_btn = gr.Button("Preguntar")

    # Define qué función se llamará cuando se haga clic en el botón 
    # y qué entradas y salidas utilizará para obtener y mostrar los resultados. 
    greet_btn.click(fn=buscar, inputs=[busqueda, gr.DataFrame(texto_emb)], outputs=output)

# Muestra la interfaz, de manera local y publica
demo.launch(share=True)


## Procesar datos de un PDF
Haremos ahora un ejemplo donde leemos un PDF para poder hacer preguntas y traer un exctracto del PDF

In [ ]:
# Antes de comenzar debemos installar Lanchain (por estudiar)
# pip install langchain pypdf
# Importamos las librerias necesarias
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
# Creamos una instancia de la clase PyPDFLoader y le proporcionamos la ruta del archivo a cargar
loader = PyPDFLoader("./mtg.pdf")

In [ ]:
# Cargamos el archivo y lo dividimos en páginas usando la instancia de PyPDFLoader
pages = loader.load_and_split()

In [ ]:
# Seleccionemos una página del contenido del PDF
pages[3].page_content

In [ ]:
# Creamos un objeto que va a hacer los cortes en el texto, cada 300 caracteres hará un salto de linea
split = CharacterTextSplitter(chunk_size=300, separator = '.\n')

In [ ]:
# Se llama a la función 'split_documents' con el argumento 'pages'
# y el resultado se guarda en la variable 'textos'
textos = split.split_documents(pages)


In [ ]:
# Imprimimos el contenido, el primero
print(textos[0].page_content)

In [ ]:
# Creamos una lista que contiene la parte de "page-content" de cada objeto en la lista "texto"
textos = [str(i.page_content) for i in textos] 
# Creamos un dataframe llamado "parrafos" a partir de la lista "texto" y nombramos la comumna "texto"
parrafos = pd.DataFrame(textos, columns=["texto"])
# Imprimimos el dataframe "parrafos" en la consola
print(parrafos)

In [ ]:
# Creamos una nueva columna llamada 'Embedding' que contiene los embeddings de los párrafos procesados con el motor 'text-embedding-ada-002'
parrafos['Embedding'] = parrafos["texto"].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))
# Guardamos el dataframe modificado en un archivo csv llamado mtg.csv
parrafos.to_csv('MTG.csv')

(Por detallar)

In [ ]:
# La misma funcion del chatbot de pregunts y respuestas
def embed_text(path="mtg.pdf"):
    conocimiento_df = pd.read_csv(path)
    conocimiento_df['Embedding'] = conocimiento_df['texto'].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))
    conocimiento_df.to_csv('mtg-embeddings.csv')
    return conocimiento_df

def buscar(busqueda, datos, n_resultados=5):
    busqueda_embed = get_embedding(busqueda, engine="text-embedding-ada-002")
    datos["Similitud"] = datos['Embedding'].apply(lambda x: cosine_similarity(x, busqueda_embed))
    datos = datos.sort_values("Similitud", ascending=False)
    return datos.iloc[:n_resultados][["texto", "Similitud", "Embedding"]]

texto_emb = parrafos
with gr.Blocks() as demo:
    busqueda = gr.Textbox(label="Buscar")
    output = gr.DataFrame(headers=['texto'])
    greet_btn = gr.Button("Preguntar")
    greet_btn.click(fn=buscar, inputs=[busqueda, gr.DataFrame(texto_emb)], outputs=output)

demo.launch()
